In [2]:
import tweepy
import shutil
import jsonpickle
import json
import datetime
import GetOldTweets3 as got

from pandas.io.json import json_normalize

import pandas as pd

In [3]:
# load Twitter API credentials
with open('../twitter_credentials.json') as cred_data:
    info = json.load(cred_data)
    consumer_key = info['CONSUMER_KEY']
    consumer_secret = info['CONSUMER_SECRET']
    access_token = info['ACCESS_TOKEN']
    access_secret = info['ACCESS_SECRET']

In [4]:
# Authenticate to Twitter
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth)

try:
    api.verify_credentials()
    print("Authentication OK")
except:
    print("Error during authentication")

Authentication OK


In [5]:
user = api.me()
print (user.name)
print(api.rate_limit_status()['resources']['search'])

Tee Moore
{'/search/tweets': {'limit': 180, 'remaining': 180, 'reset': 1564002507}}


In [6]:
my_accounts = []
for member in tweepy.Cursor(api.list_members, 'ClydeLazersex', 'evac-route-official').items():
    my_accounts.append(member.screen_name)
# Show list
my_accounts

['GDOTATL',
 'SCDOTMidlands',
 'SCDOTPeeDee',
 'SCDOTLowCountry',
 'SCDOTPiedmont',
 '511statewideva',
 'fl511_panhandl',
 '511Georgia',
 'fl511_state',
 'fl511_northeast',
 'fl511_southeast',
 'fl511_southwest',
 'fl511_tampabay',
 'fl511_central',
 '511centralva',
 '511hamptonroads',
 '511northernva',
 'NCDOT_Westmtn',
 'NCDOT_Triangle',
 'NCDOT_Piedmont',
 'NCDOT_Charlotte',
 'NCDOT_Asheville',
 'NCDOT_Scoast',
 'NCDOT_Ncoast']

In [7]:
# code modified from Jasmine Vasandani
# With the help of got.manager, get all the tweets from 10/01/2016 till 11/30/2016
# Store all the collected tweets in DF
max_tweets=3000
closure_list=[]
final_df=pd.DataFrame()
for account in my_accounts:
    tweetCriteria = got.manager.TweetCriteria().setUsername(account).setSince("2016-10-01").setUntil("2016-11-30").setMaxTweets(max_tweets)   
    print(tweetCriteria)
    tweet = got.manager.TweetManager.getTweets(tweetCriteria)
    for i in range(len(tweet)):
        tweet_dict = {}
        tweet_dict['id'] = tweet[i].id
        tweet_dict['username'] = tweet[i].username
        tweet_dict['date'] = tweet[i].date
        tweet_dict['text'] = tweet[i].text
        tweet_dict['hashtags'] = tweet[i].hashtags
        tweet_dict['geo'] = tweet[i].geo
        tweet_dict['type'] = 'official'
        closure_list.append(tweet_dict)


df1=pd.DataFrame(closure_list)
final_df = pd.concat([df1, final_df])
print("Shape of DataFrame:", final_df.shape)

Shape of DataFrame: (43016, 7)


In [8]:
# review the shape of the dataframe
final_df.shape

(43016, 7)

In [9]:
closure_list[5000]

{'id': '795743083535233024',
 'username': 'SCDOTPiedmont',
 'date': datetime.datetime(2016, 11, 7, 21, 41, 49, tzinfo=datetime.timezone.utc),
 'text': 'Accident; I-85 NB: at Exit 51A-SC146-Woodruff Rd-Greenville, no lns clsd, Ocrd: 4:40PM.| 4:41P',
 'hashtags': '',
 'geo': '',
 'type': 'official'}

In [10]:
final_df.iloc[300:350].text

300    Update: Traf congestion; I-26 WB: btwn Exit97 ...
301    Accident; I-26 WB: 2 mi W of Exit97, no lns cl...
302    Accident; I-26 WB: btwn Exit97 & 1 mi W of Exi...
303       CLEARED: Accident; I-26 WB, at Exit103.| 1:41P
304    Update: Accident; I-26 WB: at Exit103, no lns ...
305    Accident; I-26 WB: at Exit103, 2 rht lns clsd,...
306    CLEARED: Obstruction on roadway; I-126 WB, at ...
307    Update: Obstruction on roadway; I-126 WB: at G...
308    Obstruction on roadway; I-126 WB: at Greystone...
309    CLEARED: Vehicle on fire; I-26 WB, 1 mi W of E...
310    Update: Vehicle on fire; I-26 WB: 1 mi W of Ex...
311    Vehicle on fire; I-26 WB: 1 mi W of Exit106, r...
312    CLEARED: Traf congestion; I-20 WB, at Exit87.|...
313    Update: Traf congestion; I-20 WB: at Exit87, u...
314    Traf congestion; I-20 WB: at Exit87, unknown l...
315       CLEARED: Accident; I-26 WB, at Exit110.| 3:24A
316    Update: Accident; I-26 WB: at Exit110, no lns ...
317    Accident; I-26 WB: at Ex

In [11]:
# list the columns from the dataframe
final_df.columns

Index(['date', 'geo', 'hashtags', 'id', 'text', 'type', 'username'], dtype='object')

In [12]:
# list the accounts followed and the count of Tweets collected from each account
final_df.groupby(by='username')['username'].count()

username
511Georgia          364
511centralva       3000
511hamptonroads    3000
511northernva      3000
511statewideva     3000
GDOTATL             128
NCDOT_Asheville     343
NCDOT_Charlotte     877
NCDOT_Ncoast        697
NCDOT_Piedmont      137
NCDOT_Scoast        731
NCDOT_Triangle     2080
NCDOT_Westmtn        24
SCDOTLowCountry    1446
SCDOTMidlands      2324
SCDOTPeeDee         451
SCDOTPiedmont      1762
fl511_central      3000
fl511_northeast    3000
fl511_panhandl     1652
fl511_southeast    3000
fl511_southwest    3000
fl511_state        3000
fl511_tampabay     3000
Name: username, dtype: int64

In [13]:
final_df.to_csv('../data/Tweets_Historical/historic_tweets_official_07242019.csv', index = False)